In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [4]:
og = pd.read_csv("US_Accidents_March23.csv", index_col="City")
print(og.shape)
print(og.isna().sum())

(7728394, 45)
ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump                         

In [8]:
# some initial cleaning
columns_to_use = ['Severity', 'Start_Time', 'Temperature(F)', 'Wind_Chill(F)', 'Weather_Condition', 
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Humidity(%)', 
       "Start_Lat", "Start_Lng"]
# df = og[columns_to_use]
df = og

In [ ]:
Fresno = df.loc["Fresno"]         # getting 2 new dataframes, one for accidents in Fresno and one for St Louis
StLouis = df.loc["St Louis"]

Fresno = Fresno.reset_index(drop=True)
StLouis = StLouis.reset_index(drop=True)

# some cleaning first for Fresno
print(Fresno.shape)
# print(Fresno.isna().sum())

# get rid of duplicates
Fresno = Fresno.drop_duplicates(subset=['Start_Lat', 'Start_Lng', 'Start_Time'])

# fill nan values
missing = Fresno.isna().sum()
print(missing)
Fresno = Fresno.ffill().bfill()    # makes sense to forward and back fill because days that are close to each other will have similar values

# save mostly cleaned data to a smaller csv
Fresno.to_csv("Fresno_accidents.csv.zip", compression={'method': 'zip'})

(16289, 46)
Unnamed: 0               0
ID                       0
Source                   0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
End_Lat                  0
End_Lng                  0
Distance(mi)             0
Description              0
Street                   0
County                   0
State                    0
Zipcode                  0
Country                  0
Timezone                 0
Airport_Code             0
Weather_Timestamp        0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout      

In [14]:
# now similar cleaning for StLouis
print(StLouis.shape)
# print(Fresno.isna().sum())

# get rid of duplicates
StLouis = StLouis.drop_duplicates(subset=['Start_Lat', 'Start_Lng', 'Start_Time'])

# fill nan values
missing = StLouis.isna().sum()
StLouis = StLouis.ffill().bfill()    # makes sense to forward and back fill because days that are close to each other will have similar values

# save mostly cleaned data to a smaller csv
StLouis.to_csv("StLouis_accidents.csv.zip")

(1782, 45)


/var/folders/5s/rp_fb3mj71d91by4qs8qgmcr0000hs/T/ipykernel_44784/3482599675.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  StLouis = StLouis.ffill().bfill()    # makes sense to forward and back fill because days that are close to each other will have similar values


### Data Analysis below here

In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score


In [6]:
Fresno = pd.read_csv("Fresno_accidents.csv")
StLouis = pd.read_csv("StLouis_accidents.csv")
print(Fresno.columns)

Index(['Unnamed: 0', 'ID', 'Source', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Street', 'County', 'State', 'Zipcode', 'Country',
       'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Temperature(F)',
       'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)',
       'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')


In [7]:
# weather analysis for Fresno
Fresno = pd.read_csv("Fresno_accidents.csv")

new = Fresno[['Severity', 'Temperature(F)', 'Wind_Chill(F)', 'Weather_Condition', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Humidity(%)', 'Start_Lat', 'Start_Lng']]

Fresno = pd.get_dummies(new, "Weather_Condition")
print(Fresno.shape)

# get train and test data
shuffled = Fresno.sample(frac=1, random_state=21)
groups = np.array_split(shuffled, 3)

X = groups[0].to_numpy()
X_train, y_train = X[:, 1:], X[:, 0]

forest = RandomForestRegressor(oob_score=True)
forest.fit(X_train, y_train)
print(forest.oob_score_)


(16289, 42)


/usr/local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


0.08189380313712091


In [47]:
# weather analysis for Fresno without Weather_Condition
Fresno = pd.read_csv("Fresno_accidents.csv")

Fresno = Fresno[['Severity', 'Temperature(F)', 'Wind_Chill(F)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Humidity(%)']]

# Fresno = pd.get_dummies(new, "Weather_Condition")
# print(Fresno.shape)

# get train and test data
shuffled = Fresno.sample(frac=1, random_state=21)
groups = np.array_split(shuffled, 3)

X = groups[0].to_numpy()
X_train, y_train = X[:, 1:], X[:, 0]

forest = RandomForestRegressor(oob_score=True)
forest.fit(X_train, y_train)
print(f"\noob score: {forest.oob_score_}")
print(f"\nFeature importance:")
print(sorted(list(zip(forest.feature_importances_, list(Fresno.columns[1:]))))[::-1])

# test it
test = groups[1].to_numpy()
X_test, y_test = test[:, 1:], test[:, 0]
pred = forest.predict(X_test).round()   # round to compare with integers
score = accuracy_score(y_test, pred)
print(f"\nScore: {score}")


/usr/local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



oob score: 0.011902514490965577

Feature importance:
[(np.float64(0.24298072932506748), 'Pressure(in)'), (np.float64(0.22789139152106389), 'Humidity(%)'), (np.float64(0.16070237133428503), 'Wind_Chill(F)'), (np.float64(0.14939052081654153), 'Temperature(F)'), (np.float64(0.1300270366896803), 'Wind_Speed(mph)'), (np.float64(0.048221044127307926), 'Precipitation(in)'), (np.float64(0.04078690618605397), 'Visibility(mi)')]

Score: 0.9705340699815838


In [48]:
# weather analysis for St Louis without Weather_Condition
StLouis = pd.read_csv("StLouis_accidents.csv")

new = StLouis[['Severity', 'Temperature(F)', 'Wind_Chill(F)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Humidity(%)']]

# StLouis = pd.get_dummies(new, "Weather_Condition")
# print(StLouis.shape)
StLouis = new

# get train and test data
shuffled = StLouis.sample(frac=1, random_state=21)
groups = np.array_split(shuffled, 3)

X = groups[0].to_numpy()
X_train, y_train = X[:, 1:], X[:, 0]

forest = RandomForestRegressor(oob_score=True)
forest.fit(X_train, y_train)
print(f"\noob score: {forest.oob_score_}")
print(f"\nFeature importance:")
print(sorted(list(zip(forest.feature_importances_, list(Fresno.columns[1:]))))[::-1])

# test it
test = groups[1].to_numpy()
X_test, y_test = test[:, 1:], test[:, 0]
pred = forest.predict(X_test).round()
score = accuracy_score(y_test, pred)
print(f"\nScore: {score}\n")



oob score: -0.11644682327943534

Feature importance:
[(np.float64(0.2836140494717982), 'Pressure(in)'), (np.float64(0.20584241429607433), 'Humidity(%)'), (np.float64(0.1829639712960077), 'Wind_Speed(mph)'), (np.float64(0.15426288288536383), 'Temperature(F)'), (np.float64(0.1259786142944915), 'Wind_Chill(F)'), (np.float64(0.031401141850519475), 'Visibility(mi)'), (np.float64(0.015936925905744984), 'Precipitation(in)')]

Score: 0.7311072056239016



/usr/local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Again pressure and humidity were the greatest predictors and precipitation and visibility were the least


## Do I know what this is saying? What labels are they predicting and how?

In [62]:
# road features analysis for Fresno

Fresno = pd.read_csv("Fresno_accidents.csv")
new = Fresno[["Severity", 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

Fresno = new

# get test and train data
shuffled = Fresno.sample(frac=1, random_state=21)
groups = np.array_split(shuffled, 3)

X = groups[0].to_numpy()
X_train, y_train = X[:, 1:], X[:, 0]

# train a model
forest = RandomForestRegressor(oob_score=True)
forest.fit(X_train, y_train)

print(f"\noob score: {forest.oob_score_}")
print(f"\nFeature importance:")
print(sorted(list(zip(forest.feature_importances_, list(Fresno.columns[1:]))))[::-1])

# test it
test = groups[1].to_numpy()
X_test, y_test = test[:, 1:].astype(float), test[:, 0].astype(float)
pred = forest.predict(X_test).round()
score = accuracy_score(y_test, pred)
print(f"\nScore: {score}\n")



oob score: -0.0011846518866385214

Feature importance:
[(np.float64(0.3806420466837095), 'Crossing'), (np.float64(0.22519064318584509), 'Junction'), (np.float64(0.18156558884674198), 'Railway'), (np.float64(0.15352899914044696), 'Traffic_Signal'), (np.float64(0.05279558774504897), 'Stop'), (np.float64(0.003606462537267325), 'Amenity'), (np.float64(0.0014289338053281538), 'Station'), (np.float64(0.0012417380556119769), 'No_Exit'), (np.float64(0.0), 'Turning_Loop'), (np.float64(0.0), 'Traffic_Calming'), (np.float64(0.0), 'Roundabout'), (np.float64(0.0), 'Give_Way'), (np.float64(0.0), 'Bump')]

Score: 0.9758747697974217



/usr/local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [63]:
# road features analysis for StLouis

StLouis = pd.read_csv("StLouis_accidents.csv")
new = StLouis[["Severity", 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

StLouis = new

# get test and train data
shuffled = StLouis.sample(frac=1, random_state=21)
groups = np.array_split(shuffled, 3)

X = groups[0].to_numpy()
X_train, y_train = X[:, 1:], X[:, 0]

# train a model
forest = RandomForestRegressor(oob_score=True)
forest.fit(X_train, y_train)

print(f"\noob score: {forest.oob_score_}")
print(f"\nFeature importance:")
print(sorted(list(zip(forest.feature_importances_, list(StLouis.columns[1:]))))[::-1])

# test it
test = groups[1].to_numpy()
X_test, y_test = test[:, 1:].astype(float), test[:, 0].astype(float)
pred = forest.predict(X_test).round()
score = accuracy_score(y_test, pred)
print(f"\nScore: {score}\n")



oob score: -0.016420991046837363

Feature importance:
[(np.float64(0.303074518765442), 'Stop'), (np.float64(0.22882838225069915), 'Traffic_Signal'), (np.float64(0.1859380462573112), 'Station'), (np.float64(0.13207339535748586), 'Junction'), (np.float64(0.09131038250415667), 'Give_Way'), (np.float64(0.03026211642418465), 'Crossing'), (np.float64(0.013628809218238138), 'No_Exit'), (np.float64(0.011317951077513284), 'Amenity'), (np.float64(0.0035663981449690733), 'Railway'), (np.float64(0.0), 'Turning_Loop'), (np.float64(0.0), 'Traffic_Calming'), (np.float64(0.0), 'Roundabout'), (np.float64(0.0), 'Bump')]

Score: 0.7943760984182777



/usr/local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
